Here we implement the boundary operator, as well as the combinatorical Laplacian.

In [47]:
import numpy as np
import itertools as it

In [56]:
def boundary_operator(x_tuple):
    x_np = np.array(x_tuple)
    indices = np.nonzero(x_np)[0]
    dictionary = {}
    for i in range(len(indices)):
        helper = x_np.copy()
        helper[indices[i]] = 0
        dictionary[tuple(helper)] = (-1)**i
    return dictionary

def boundary_operator_dict(N_vertices):
    dictionary = {}
    for b in it.product(range(2), repeat=N_vertices):
        helper = boundary_operator(b)
        for key in helper.keys():
            dictionary[(tuple(b),key)] = helper[key]
    return dictionary

In [73]:
d = boundary_operator_dict(3)
d

{((0, 0, 1), (0, 0, 0)): 1,
 ((0, 1, 0), (0, 0, 0)): 1,
 ((0, 1, 1), (0, 0, 1)): 1,
 ((0, 1, 1), (0, 1, 0)): -1,
 ((1, 0, 0), (0, 0, 0)): 1,
 ((1, 0, 1), (0, 0, 1)): 1,
 ((1, 0, 1), (1, 0, 0)): -1,
 ((1, 1, 0), (0, 1, 0)): 1,
 ((1, 1, 0), (1, 0, 0)): -1,
 ((1, 1, 1), (0, 1, 1)): 1,
 ((1, 1, 1), (1, 0, 1)): -1,
 ((1, 1, 1), (1, 1, 0)): 1}